##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# RNN을 이용한 텍스트 생성(즉시 실행)

<table class="tfo-notebook-buttons" align="left">
<td>
<a target="_blank" href="https://www.tensorflow.org/tutorials/sequences/text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
</td><td>
<a target="_blank"  href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/sequences/text_generation.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>  
</td><td>
<a target="_blank"  href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/sequences/text_generation.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a></td></table>

Note: 이 문서는 텐서플로 커뮤니티에서 번역했습니다. 커뮤니티 번역 활동의 특성상 정확한 번역과 최신 내용을 반영하기 위해 노력함에도
불구하고 [공식 영문 문서](https://www.tensorflow.org/?hl=en)의 내용과 일치하지 않을 수 있습니다.
이 번역에 개선할 부분이 있다면
[tensorflow/docs](https://github.com/tensorflow/docs) 깃헙 저장소로 풀 리퀘스트를 보내주시기 바랍니다.
문서 번역이나 리뷰에 지원하려면 [이 양식](https://bit.ly/tf-translate)을
작성하거나
[docs@tensorflow.org](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs)로
메일을 보내주시기 바랍니다.

In [ ]:
!pip install tf-nightly

이 튜토리얼은 철자기반의 RNN을 이용해서 어떻게 텍스트를 생성할 수 있는지를 설명한다. Andrej Karpathy의 [반복신경망의 비합리적 효과](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)셰익스피어의 작품 데이터셋을 가지고 작업을 할 예정이다. 이 데이터(셰익스피어)에서 나오는 일련의 문자들("e")을 고려하여, 그 순서에서 다음 문자를 예측하기 위해서 모델을 학습시킨다. 모델을 반복적으로 호출함으로써 더 긴 텍스트 시퀀스가 생성할 수 있다.

Note: GPU 가속을 활성화하여 이 노트북을 더 빠르게 실행할 수 있다. In Colab: *Runtime > Change runtime type > Hardware acclerator > GPU*. 만약 로컬환경에서 실행한다면, TensorFlow 버전이 1.11 이상인지 확인해야 한다.

이 튜토리얼은 [tf.keras](https://www.tensorflow.org/programmers_guide/keras)와 [eager excution](https://www.tensorflow.org/programmers_guide/eager)(즉시실행)로 구현된 실행 가능한 코드를 포함한다. 이 튜토리얼의 모델이 30 에포크(epoch) 동안 학습하고, 문자 "Q"로 시작했을때 나온 샘플 출력이다:

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

일부 문장은 문법적이지만, 대부분은 말이 안 되는 문장이다. 모델은 단어의 의미를 학습해오지 않았다, 그러나 다음 사항들을 고려해보자:

* 모델은 철자 기반이다. 학습을 시작할 때, 모델은 영어 단어의 철자를 어떻게 쓰는지 알지 못하거나 심지어 단어들이 텍스트의 단위라는 것을 알지 못했다.

* 출력의 구조는 연극의 대본과 유사하다 - 텍스트 블록이 대체적으로 데이터셋과 유사하게 모두 대문자로된 화자의 이름으로 시작한다.

* 아래에서 증명된 바와 같이 모델은 작은 텍스트 묶음(각각 100자)에 대해 학습되며, 여전히 일관된 구조를 가진 긴 텍스트 시퀀스를 생성할 수 있다.

## 설정

### 텐서플로우 및 다른 라이브러리 불러오기

In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

### 셰익스피어 데이터셋 다운로드

자신의 데이터에서 이 코드를 실행하려면 다음 행을 변경하십시오.(셰익스피어 외의 개인 데이터)

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


### 데이터 읽기
먼저, 본문을 보자.

In [5]:
# Read, then decode for py2 compat. / 읽은 다음, py2 호환을 위해 디코딩한다.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it  / 텍스트의 길이는 그 안에 있는 문자 수이다.
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [6]:
# Take a look at the first 250 characters in text  /  텍스트의 처음 250자를 보자.
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [7]:
# The unique characters in the file / 파일에 있는 고유한 문자
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


## 텍스트 처리

### 텍스트 벡터화
학습을 이전에 우리는 문자열을 대표 수치값과 연결(사상)시켜야 한다. 두 개의 검색 테이블 생성: 하나는 문자를 숫자에 연결시키기 위함이고, 다른 하나는 숫자를 문자에 연결시키기 위함이다.

In [8]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

이제 우리는 각 문자에 대한 정수 표현을 가지고 있다. 0부터 `len(unique)`까지의 인덱스로 문자를 매핑(연결, 사상)한 것에 주의한다.

In [9]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [10]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


### 예측 작업

철자 하나 혹은 일련의 철자들을 고려했을 때, 가장 그럴듯한 다음 철자는 무엇인가? 이것은 우리가 (예측을) 수행하기 위해 모델을 학습시키는 작업이다. 모델에 대한 입력은 일련의 철자들이고, 우리는 각 단계마다 다음 문자, 즉 출력을 예측하기 위해 모델을 학습시킨다.

RNN은 이전에 본 요소에 의존하는 내부 상태를 유지하므로, 이 순간까지 연산된 모든 문자를 고려하면, 다음 문자는 무엇인가?

### 학습 예제 및 타겟(목표변수) 생성

다음으로 텍스트를 예제 시퀀스로 나눠라. 각 입력 시퀀스는 텍스트로 부터 얻은 문자들의 `seq_length`를 포함한다. 
한 문자를 오른쪽으로 이동하지 않는 한, 해당 대상에는 동일한 길이의 텍스트가 포함되어 있다.